This notebook focuses on integrating food delivery data from multiple sources and analyzing the combined dataset. The data is merged from CSV, JSON, and SQL formats using appropriate join keys, and the final dataset is used as the single source of truth for all analysis. The objective is to derive meaningful insights related to orders, users, revenue, and restaurant performance.

Import Libraries and Data Sets

In [1]:
import pandas as pd

orders = pd.read_csv("/content/orders.csv")


In [2]:
users = pd.read_json("/content/users.json")


In [4]:
import sqlite3

conn = sqlite3.connect(":memory:")

with open("restaurants.sql", "r") as f:
    sql_script = f.read()

conn.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)


In [5]:
# Step 4: Merge Orders with Users (LEFT JOIN)
orders_users = orders.merge(
    users,
    on="user_id",
    how="left"
)


In [6]:
# Step 5: Merge with Restaurants (LEFT JOIN)
final_df = orders_users.merge(
    restaurants,
    on="restaurant_id",
    how="left"
)


In [7]:
# Step 6: Rename duplicate columns (if needed)
final_df.rename(columns={
    "city_x": "user_city",
    "city_y": "restaurant_city"
}, inplace=True)


In [40]:
# Step 7: Export Final Dataset
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [41]:
import pandas as pd

df = pd.read_csv("final_food_delivery_dataset.csv")


In [43]:
df.columns.tolist()


['order_id',
 'user_id',
 'restaurant_id',
 'order_date',
 'total_amount',
 'restaurant_name_x',
 'name',
 'city',
 'membership',
 'restaurant_name_y',
 'cuisine',
 'rating']

In [44]:
# Fix amount column
if "order_amount" not in df.columns and "total_amount" in df.columns:
    df["order_amount"] = df["total_amount"]

# Fix user city column
if "user_city" not in df.columns and "city" in df.columns:
    df["user_city"] = df["city"]


In [46]:
# Inspect columns
df.columns.tolist()


['order_id',
 'user_id',
 'restaurant_id',
 'order_date',
 'total_amount',
 'restaurant_name_x',
 'name',
 'city',
 'membership',
 'restaurant_name_y',
 'cuisine',
 'rating',
 'order_amount',
 'user_city']

In [47]:
# Normalize column names safely (one-time fix)

# amount
if "order_amount" not in df.columns:
    if "total_amount" in df.columns:
        df["order_amount"] = df["total_amount"]

# user city
if "user_city" not in df.columns:
    if "city" in df.columns:
        df["user_city"] = df["city"]

# order date
if "order_date" not in df.columns:
    if "date" in df.columns:
        df["order_date"] = df["date"]


Multiple Choice Answers

1. Which city has the highest total revenue (total_amount) from Gold members?  

In [48]:
df[df["membership"] == "Gold"] \
  .groupby("user_city")["order_amount"] \
  .sum() \
  .sort_values(ascending=False)


,order_amount
user_city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


2. Which cuisine has the highest average order value across all orders?

In [49]:
df.groupby("cuisine")["order_amount"] \
  .mean() \
  .sort_values(ascending=False)


,order_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


3. How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?

In [50]:
df.groupby("user_id")["order_amount"] \
  .sum() \
  .gt(1000) \
  .sum()


np.int64(2544)

4. Which restaurant rating range generated the highest total revenue?

In [51]:
import pandas as pd

df["rating_range"] = pd.cut(
    df["rating"],
    bins=[3.0, 3.5, 4.0, 4.5, 5.0],
    labels=["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]
)

df.groupby("rating_range")["order_amount"] \
  .sum() \
  .sort_values(ascending=False)


/tmp/ipython-input-3346223694.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["order_amount"] \


,order_amount
rating_range,
4.6–5.0,2197030.75
4.1–4.5,1960326.26
3.0–3.5,1881754.57
3.6–4.0,1717494.41


5. Among Gold members, which city has the highest average order value?

In [52]:
df[df["membership"] == "Gold"] \
  .groupby("user_city")["order_amount"] \
  .mean() \
  .sort_values(ascending=False)


,order_amount
user_city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


6. Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?

In [54]:
df.groupby("cuisine").agg(
    restaurants=("restaurant_name_y", "nunique"),
    revenue=("order_amount", "sum")
).sort_values("restaurants")

,restaurants,revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


7. What percentage of total orders were placed by Gold members? (Rounded to nearest integer)

In [55]:
round(
    (df[df["membership"] == "Gold"].shape[0] / df.shape[0]) * 100
)


50

8. Which restaurant has the highest average order value but less than 20 total orders?

In [63]:
# find restaurant name column
possible_cols = [c for c in df.columns if "rest" in c.lower() and "id" not in c.lower()]
print("Restaurant column detected:", possible_cols)

rest_col = possible_cols[0]  # take first match

restaurant_stats = (
    df.groupby(rest_col)
      .agg(
          total_orders=("order_id", "count"),
          avg_order_value=("total_amount", "mean")
      )
      .reset_index()
)

filtered = restaurant_stats[
    restaurant_stats["total_orders"] < 20
]

result = filtered.sort_values(
    "avg_order_value",
    ascending=False
)

result.head()

Restaurant column detected: ['restaurant_name_x', 'restaurant_name_y']


,restaurant_name_x,total_orders,avg_order_value
173,Hotel Dhaba Multicuisine,13,1040.222308
354,Sri Mess Punjabi,12,1029.180833
236,Ruchi Biryani Punjabi,16,1002.140625
343,Sri Delights Pure Veg,18,989.467222
88,Classic Kitchen Family Restaurant,19,973.167895


9. Which combination contributes the highest revenue?

In [60]:
# 9. Combination with highest revenue
# -------------------------------
# Filter only valid MCQ combinations
mcq_combinations = df[
    ((df["membership"] == "Gold") & (df["cuisine"].isin(["Indian", "Italian"]))) |
    ((df["membership"] == "Regular") & (df["cuisine"].isin(["Indian", "Chinese"])))
]

# Calculate revenue
combo_revenue = (
    mcq_combinations
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .reset_index()
    .sort_values("total_amount", ascending=False)
)

# Top combination (MCQ answer)
top_combo = combo_revenue.iloc[0]

print("Highest revenue combination:")
print(f"{top_combo['membership']} + {top_combo['cuisine']}")

Highest revenue combination:
Gold + Italian


10. During which quarter of the year is the total revenue highest?

In [59]:
df["order_date"] = pd.to_datetime(df["order_date"])
df["quarter"] = df["order_date"].dt.to_period("Q")

df.groupby("quarter")["order_amount"] \
  .sum() \
  .sort_values(ascending=False)


/tmp/ipython-input-6754768.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["order_date"] = pd.to_datetime(df["order_date"])


,order_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


Numerical Answers

1. How many total orders were placed by users with Gold membership?

In [64]:
df[df["membership"] == "Gold"]["order_id"].nunique()


4987

2. What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?

In [65]:
round(
    df[df["user_city"] == "Hyderabad"]
    .groupby("order_id")["order_amount"]
    .sum()
    .sum()
)


1889367

3. How many distinct users placed at least one order?

In [66]:
df["user_id"].nunique()


2883

4. What is the average order value (rounded to 2 decimals) for Gold members?

In [67]:
round(
    df[df["membership"] == "Gold"]
    .groupby("order_id")["order_amount"]
    .sum()
    .mean(),
    2
)


np.float64(797.15)

5. How many orders were placed for restaurants with rating ≥ 4.5?

In [68]:
df[df["rating"] >= 4.5]["order_id"].nunique()


3374

6. How many orders were placed in the top revenue city among Gold members only?

In [71]:
# Find top revenue city for Gold members
top_gold_city = (
    df[df["membership"] == "Gold"]
    .groupby("user_city")["order_amount"]
    .sum()
    .idxmax()
)

# Count orders in that city (Gold only)
df[(df["membership"] == "Gold") & (df["user_city"] == top_gold_city)]["order_id"].nunique()

1337

In [72]:
# Load dataset
import pandas as pd
df = pd.read_csv("final_food_delivery_dataset.csv")


Fill in the Blanks

1. The column used to join orders.csv and users.json is __________.

In [73]:
# 1) Column used to join orders.csv and users.json
# (common key between orders and users)
{"orders_users_join_key": "user_id"}


{'orders_users_join_key': 'user_id'}

2. The dataset containing cuisine and rating information is stored in __________ format.

In [74]:
# 2) Dataset format containing cuisine and rating information
{"restaurant_master_format": "SQL"}


{'restaurant_master_format': 'SQL'}

3. The total number of rows in the final merged dataset is __________.


In [75]:
# 3) Total number of rows in the final merged dataset
df.shape[0]


10000

4. If a user has no matching record in users.json, the merged values will be __________.

In [76]:
# 4) Values when a user has no matching record in users.json
# (demonstrate by checking missing membership/user fields)
df[["membership"]].isna().any().to_dict()


{'membership': False}

5.The Pandas function used to combine datasets based on a key is __________.

In [77]:
# 5) Pandas function used to combine datasets based on a key
{"pandas_join_function": "merge()"}


{'pandas_join_function': 'merge()'}

6. The column membership in the final dataset originates from the __________ file.  

In [78]:
# 6) File from which 'membership' column originates
{"membership_source_file": "users.json"}


{'membership_source_file': 'users.json'}

7. The join key used to combine orders data with restaurant details is __________.  

In [79]:
# 7) Join key used to combine orders with restaurant details
{"orders_restaurants_join_key": "restaurant_id"}


{'orders_restaurants_join_key': 'restaurant_id'}

8. The column that helps identify the type of food served by a restaurant is __________.  

In [80]:
# 8) Column identifying the type of food served by a restaurant
{"food_type_column": "cuisine"}


{'food_type_column': 'cuisine'}

9. If a user places multiple orders, their personal details appear __________ times in the final merged dataset.  

In [82]:
# 9) How many times user details appear if a user places multiple orders
# (verify by comparing orders vs users grain)
orders_per_user = df.groupby("user_id")["order_id"].nunique()
{"user_details_appearance": "multiple" if (orders_per_user > 1).any() else "once"}


{'user_details_appearance': 'multiple'}